In [ ]:
import glob
import pandas as pd

## 차분값, 이동평균, MACD 함수 계산 후 csv저장

In [ ]:
# MACD 계산 함수
def calculate_macd(data, col_name, short_window=3, long_window=5, signal_window=9):
    short_ema = data[col_name].ewm(span=short_window, adjust=False).mean()
    long_ema = data[col_name].ewm(span=long_window, adjust=False).mean()
    
    data[f'{col_name}_macd'] = short_ema - long_ema
    data[f'{col_name}_signal'] = data[f'{col_name}_macd'].ewm(span=signal_window, adjust=False).mean()
    data[f'{col_name}_macd_hist'] = data[f'{col_name}_macd'] - data[f'{col_name}_signal']
    
    return data

In [ ]:
# HOURLY_로 시작하는 모든 .csv 파일 경로 찾기
file_paths = glob.glob('/data/ephemeral/home/Jinwook/project/level1-classificationinmachinelearning-recsys-05/data2 copy/HOURLY_MARKET-close_pred.csv')  # 경로를 실제 디렉토리로 변경하세요.

# 모든 파일 처리
for file in file_paths:
    # CSV 파일 읽기
    df = pd.read_csv(file)
    
    # datetime column을 제외한 모든 칼럼에 대해 현재값 - 이전값 계산
    for col in df.columns:
        if col != 'datetime':  # datetime column 제외
            df[f'{col}_diff'] = df[col].diff()  # 현재값 - 이전값
            df[f'{col}_3h_ma'] = df[col].rolling(window=3).mean()

            # MACD 계산
            df = calculate_macd(df, col_name=col)
    
    # 수정된 DataFrame을 새로운 파일로 저장
    df.to_csv(file, index=False)

## 차분값의 변동율 추종 여부 계산

In [ ]:
# 데이터 로드
data = pd.read_csv('/data/ephemeral/home/Jinwook/project/level1-classificationinmachinelearning-recsys-05/data2 copy/HOURLY_MARKET-close_pred_with_ma_diff.csv')
data1 = pd.read_csv('/data/ephemeral/home/Jinwook/project/level1-classificationinmachinelearning-recsys-05/data2 copy/HOURLY_MARKET-close_with_ma_diff.csv')


# 차분값의 부호가 같은지 여부 확인 (True/False)
data['same_sign'] = (data['close_3h_ma_diff'] * data1['close_pred_3h_ma_diff']) > 0

# True의 비율 계산
true_ratio = data['same_sign'].mean()

# 결과 출력
print(f"True 비율: {true_ratio:.2%}")